In [76]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd

In [77]:
# Chrome options to fix (unknown error: DevToolsActivePort file doesn't exist)
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [78]:
def get_job_info(location, role, num_jobs):
    driver = webdriver.Chrome()
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + role + '"&locKeyword=United%20States&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=false&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []
    cities_states = []
    salary_range = []
    n = 1
    date_posted_button = driver.find_element_by_xpath('//*[@id="filter_fromAge"]')
    date_posted_button.click()
    
    # Select Date Posted = 2 weeks
    date_range_object = driver.find_element_by_xpath('//*[@id="PrimaryDropdown"]/ul/li[5]')
    date_range_object.click()
    
    #while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.
    # 
    while n < (num_jobs + 1):
        time.sleep(3)
        job_button = driver.find_element_by_xpath(f"//*[@id='MainCol']/div[1]/ul/li[{n}]")
        job_button.click()
        
        # Attempt to close Sign Up Pop Up if any
        try:
            driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()
            print('closed sign up page')
        except:
            pass
        
        # Gather necessary information
        try:
            city_state_tag = driver.find_element_by_xpath(f"//*[@id='MainCol']/div[1]/ul/li[{n}]/div[2]/div[2]/span")
            salary_range_tag = driver.find_element_by_xpath(f"//*[@id='MainCol']/div[1]/ul/li[{n}]/div[2]/div[3]/div[1]/span")
            job_description_tag = driver.find_elements_by_class_name("jobDescriptionContent desc")

        except:
            pass
            
        print(city_state_tag.text)
        print(salary_range_tag.text)
        print(len(job_description_tag))
        
        
        n += 1

In [74]:
#This line will open a new chrome window and start the scraping.
get_job_info('United States', "data scientist", 5)

closed sign up page
Rockville, MD
$72K - $120K (Glassdoor est.)
0
closed sign up page
Portland, ME
$65K - $105K (Glassdoor est.)
0
Atlanta, GA
$100K - $161K (Glassdoor est.)
0
Lorton, VA
$120K (Glassdoor est.)
0
Dallas, TX
$102K - $122K (Glassdoor est.)
0
